## Chat Customer Service

In [1]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

True

In [2]:
## define the output parser
from typing import  Optional
from pydantic import BaseModel, Field
from langchain_core.output_parsers import JsonOutputParser

class OrderQuery(BaseModel):
    order_id: Optional[str] = Field(description="The id of the order")
    product_name: Optional[str] = Field(description="The name of the product")
    issue_type: str = Field(description="The type of the issue like: refund, replacement, recommendation etc. Mandatory")
    details: str = Field(description="The details of the issue. Mandatory")

parser = JsonOutputParser(pydantic_object=OrderQuery)

In [3]:
## define the prompt

from langchain_openai import ChatOpenAI
llm = ChatOpenAI(temperature=0, model_name="gpt-4o-mini")

In [4]:
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import SQLChatMessageHistory

def get_session_history(session_id):
    return SQLChatMessageHistory(session_id, "sqlite:///chat_history.db")

In [8]:
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, MessagesPlaceholder, PromptTemplate, SystemMessagePromptTemplate

system = SystemMessagePromptTemplate.from_template("You are a support assistant.")

order_query = HumanMessagePromptTemplate.from_template(
    template='''
    You need to parse the user query. Here is your formatting instruction: {format_instruction}
    Query: {query}''',
    partial_variables={'format_instruction': parser.get_format_instructions()}
)

messages = [system, MessagesPlaceholder(variable_name='history'), order_query]
prompt = ChatPromptTemplate(messages=messages)
chain = prompt | llm | parser 
runnable_with_history = RunnableWithMessageHistory(chain, get_session_history, 
                                                   input_messages_key='query', 
                                                   history_messages_key='history')



In [9]:
requests = []

def chat_with_llm(session_id, input):
    output = runnable_with_history.invoke(
        {'query': input},
        config={'configurable': {'session_id': session_id}}
    )
    requests.append(output)

In [13]:
user_id = "test"
chat_with_llm(user_id, "reccomend me the product for for skin brightening")
chat_with_llm(user_id, "give me the status of my order 743")
chat_with_llm(user_id, "I ordered creme and my parcel haven't arrived. the order num 99898")

Error in RootListenersTracer.on_chain_end callback: KeyError('output')
Error in RootListenersTracer.on_chain_end callback: KeyError('output')
Error in RootListenersTracer.on_chain_end callback: KeyError('output')


In [12]:
print(requests)

[{'order_id': None, 'product_name': None, 'issue_type': 'recommendation', 'details': 'Recommend a product for skin brightening.'}, {'order_id': None, 'product_name': None, 'issue_type': 'status inquiry', 'details': 'Requesting the status of the order.'}, {'order_id': None, 'product_name': None, 'issue_type': 'lost', 'details': 'my package lost'}]
